In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# This notebook is a modified copy of 
# https://www.kaggle.com/nickdudaev/talkingdata-mobile-user-demographics/categorizing-labels
# unfortunately the fork and download didn't work

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re


# Original author's foreword

## Foreword:

I realize that there maybe some mistakes in generalizing labels. 
I also have left some categories unchanged because they have big counts and because I am not sure how to categorize them.
For example, *_Tencent_* label. I googled and they produce different apps, messangers, games and so on. How to categorize them
Another example, *_Industry tag_*. I left it unchanged.

Regarding minor mistakes in categorization I thought I will not care much about it at the moment because:
* in general labels list is very messy. I have strong impression that these was generated from few disparate lists which makes it difficult to work with in a more reasonable way
* Few mistakes in categorization should not have a big impact on final picture when I look at it from *_gender age_* angle

Further comments are for each category, those explain my logic

I also tried to avoid creating too 'narrow' categories because it could be that this category would not be that representative

** P.S. This can be by no doubt further improved **

In [3]:
labels = pd.read_csv("files/label_categories.csv")
app_labels = pd.read_csv("files/app_labels.csv")
apps = pd.merge(app_labels, labels, how='left', on='label_id')

In [4]:
gp = app_labels.groupby("app_id")

In [5]:
len(app_labels.label_id.unique())

507

In [6]:
df=pd.DataFrame(app_labels.label_id.unique())

In [7]:
df.columns=["label_id"]

In [8]:
df["label_id"]=np.sort(df.label_id)

In [9]:
uniq_lab = df.merge(labels, on="label_id", how="left")

In [10]:
uniq_lab.to_csv("files/unique_categories.csv", index=False)

In [11]:
def to_Games(x):
    # I have gone through 'almost' manually through existing categories and came up with this non-elegant regex
    # there are a lot of games, so we can split the games category further
    # add gambling
    if x in ["game-game type"]:
        return('cat_Game_Other')
    if re.search("(?i)((poker)|(gambl)|(roulette)|(casino))"
                 ,x) is not None:
        return('cat_Game_Gambling')
    # add rpg and strategy games
    if re.search('(RPG)|(MMO)|(ARPG)|([sS]trategy)|(Warcraft)|(dota)', x) is not None:
        return('cat_Game_RPG')
    if re.search("(?i)((tower)|(role))", x) is not None:
        return('cat_Game_RPG')
    # add sport games, be aware of activity apps
    pattern = re.compile("(?i)((sport)|(game)|(fishing)|(box)|(motorcycle)|(parkour)|(flight))") 
    if re.search("(?i)((ball)|(tennis)|(billard)|(sailing)|(racing)|(rowing)|(aircraft))"
                 ,x) is not None or len(pattern.findall(x))==2:
        return('cat_Game_Sports')
    # puzzle and skill
    pattern = re.compile("(?i)((gem)|(game)|(table)|(board)|(card)|(word)|(education)|(adventure))") 
    if re.search('(?i)((chess)|(puzz)|(math)|(dice)|(simulat)|(tactic)|(majiang))', x) is not None or len(pattern.findall(x))>1:
        return('cat_Game_Skill')
    if re.search('(?i)((cutting)|(fault)|(answer))', x) is not None or len(pattern.findall(x))>1:
        return('cat_Game_Skill')
    # add action games
    pattern = re.compile("(?i)((action)|(game))")
    if re.search("(?i)((shoot)|(fight)|(tank)|(snipe)|(zombie)|(military)|(moba)|(martial)|(senki)|(ninja))"
                 ,x) is not None or len(pattern.findall(x))==2:
        return('cat_Game_Action')
    # then just categorise other games
    if re.search("(?i)(game)|(farm)|(cosplay)",x) is not None:
        return('cat_Game_Other')
    else:
        return(x)

In [12]:
def to_Finance(x):
    if re.search("(?i)((financ)|(profit)|(income)|(risk)|(bank)|(insurance)|(cost)|(metals))", x) is not None:
        return("cat_Finance")
    if re.search("(?i)((exchange)|(credit)|(bank)|(fund)|(trust)|(futures)|(liquid))", x) is not None:
        return('cat_Finance')
    if re.search("(?i)((stock)|(shares)|(account)|(wealth)|(loan)|(securities)|(imf))", x) is not None:
        return('cat_Finance')
    else:
        return(x)

In [13]:
def to_Family(x):
    if re.search("(?i)((parent)|(family)|(bab)|(child)|(fetus)|(pregnant))", x) is not None:
        return("cat_Family")
    if re.search("(?i)((relatives)|(love)|(marry)|(paternity)|(mother))", x) is not None:
        return("cat_Family")
    if re.search("(?i)((raising))", x) is not None:
        return("cat_Family") #these might be games, but probably better in family category
    else:
        return(x)

In [14]:
def to_Shopping(x):
    if re.search("(?i)((suitcase)|(shoe))", x) is not None:
        return("cat_Shopping")
    if re.search("(?i)((shopping)|(buy)|(pay)|(sell)|(mall)|(coupon)|(amoy))", x) is not None:
        return("cat_Shopping")
    else:
        return(x)

def to_Beauty(x):
    if re.search("(?i)((beauty)|(luxury)|(make)|(fashion)|(nail)|(hair)|(skin))", x) is not None:
        return("cat_Beauty")
    else:
        return(x)

In [15]:
def to_AsIs(x):
    '''
    Large categories will be kept as they are.
    '''
    if x in ["Industry tag", "Custom label", "Tencent", "unknown"]:
        return("cat_"+x)
    else:
        return(x)

In [23]:
def to_Entertainment(x):
    if re.search("(?i)((comic)|(movie)|(show)|(video)|(TV)|(film))", x) is not None:
        return("cat_Entertainment")
    if re.search("(?i)((cool)|(fun)|(music)|(animation)|(fiction)|(wars)|(entertainment))", x) is not None:
        return("cat_Entertainment")
    if re.search("(?i)((joke))", x) is not None:
        return("cat_Entertainment")
    else:
        return(x)

def to_Culture(x):
    if re.search("(?i)((culture)|(antique)|(literature)|(poetry)|(paint)|(photo)|(radio)|(xian)|(yong))", x) is not None:
        return("cat_Culture")
    else:
        return(x)

In [18]:
def to_Property(x):
    if re.search("(?i)((property)|(estate))", x) is not None:
        return("cat_Property")
    else:
        return(x)

In [17]:
def to_Communication(x):
    if re.search("(?i)((IM)|(commu)|(email)|(phone)|(blogs)|(sharing)|(express))", x) is not None:
        return("cat_Communication")
    if re.search("(?i)((contact)|(weibo))", x) is not None:
        return("cat_Communication")
    else:
        return(x)

In [19]:
def to_Travel(x):
    if re.search("(?i)((aviatio)|(air)|(booking)|(travel)|(hotel)|(navigation))", x) is not None:
        return("cat_Travel")
    if re.search("(?i)((train)|(taxi)|(map)|(reserva)|(accom)|(bus)|(car)|(flight))", x) is not None:
        return("cat_Travel")
    if re.search("(?i)((journey)|(transport)|(tourism)|(mobility)|(region)|(rental))", x) is not None:
        return("cat_Travel")
    if re.search("(?i)((tour))", x) is not None:
        return("cat_Travel")
    else:
        return(x)

In [20]:
def to_Education(x):
    # merge information, technology and education categories
    if re.search("(?i)((stud)|(language)|(educ)|(exams)|(lib)|(dict))", x) is not None:
        return("cat_Education") # former education
    if re.search("(?i)((read)|(psychology)|(book)|(novel))", x) is not None:
        return("cat_Education") # former education
    else:
        return(x)
    
def to_Information(x):
    if re.search("(?i)((news)|(information)|(magazine)|(review)|(peace))", x) is not None:
        return("cat_Information") # former information
    else:
        return(x)
    
def to_Technology(x):
    if re.search("(?i)((weather)|(wifi)|(science)|(tech)|(ping)|(smart))", x) is not None:
        return("cat_Education") # former technology
    if re.search("(?i)((engin)|(store)|(system)|(hardware)|(P2P)|(network))", x) is not None:
        return("cat_Education") # former technology
    else:
        return(x)

In [21]:
def to_Health(x):
    if re.search("(?i)((sport)|(parkour)|(health)|(medic)|(weight)|(recipes)|(activ))", x) is not None:
        return("cat_Health")
    if re.search("(?i)((order)|(vita)|(natur)|(doc)|(enthu)|(exercise)|(effort)|(vermice))", x) is not None:
        return("cat_Health")
    else:
        return(x)

In [22]:
def to_Productivity(x):
    if re.search("(?i)((manage)|(prof)|(clock)|(calendar)|(jobs)|(schedu))", x) is not None:
        return("cat_Productivity")
    if re.search("(?i)((note)|(edit)|(util)|(search)|(browser)|(desktop))", x) is not None:
        return("cat_Productivity")
    if re.search("(?i)((customiz)|(effective)|(pursu))", x) is not None:
        return("cat_Productivity")
    else:
        return(x)

In [24]:
def to_Esoteric(x):
    if re.search("(?i)((church)|(lottery)|(heritage)|(romance)|(myth)|(magic))", x) is not None:
        return("cat_Esoteric")
    if re.search("(?i)((fantasy)|(ident)|(horoscope))", x) is not None:
        return("cat_Esoteric")
    else:
        return(x)

In [25]:
def to_House(x):
    if re.search("(?i)((hous)|(deco)|(appl)|(cozy)|(comfort)|(liv)|(furn))", x) is not None:
        return("cat_House")
    else:
        return(x)

In [26]:
def to_Similar(x):
    if re.search("(?i)(free)", x) is not None:
        return("cat_Free")
    if re.search("(?i)(service)", x) is not None:
        return("cat_Service")
    else:
        return(x)

In [27]:
def to_Category(x):
    '''
    The idea of this prioritisation of categories is 
    that some label have stronger separation power than others.
    E.g.: if an app is categorised as 'child puzzle', the fact that 
    this app is made for children is regarded more important than the 
    fact that it is a puzzle game.
    
    All new categories start with `cat_` so that the category string 
    is changed, when categorised by a function.
    '''    
    initial = x
    cond = lambda x: x==initial
    cat=to_AsIs(x)
    if(cond(cat)):
        cat=to_Family(x)
    if(cond(cat)):
        cat=to_Beauty(x)
    if(cond(cat)):
        cat=to_Finance(x)
    if(cond(cat)):
        cat=to_Shopping(x)
    if(cond(cat)):
        cat=to_Games(x)
    if(cond(cat)):
        cat=to_Travel(x)
    if(cond(cat)):
        cat=to_Entertainment(x)
    if(cond(cat)):
        cat=to_Culture(x)    
    if(cond(cat)):
        cat=to_Communication(x)
    if(cond(cat)):
        cat=to_Property(x)
    if(cond(cat)):
        cat=to_Information(x)
    if(cond(cat)):
        cat=to_Technology(x)    
    if(cond(cat)):
        cat=to_Education(x)
    if(cond(cat)):
        cat=to_Health(x)
    if(cond(cat)):
        cat=to_Productivity(x)
    if(cond(cat)):
        cat=to_Esoteric(x)
    if(cond(cat)):
        cat=to_House(x)
    if(cond(cat)):
        cat=to_Similar(x)
    if(cond(cat)):
        cat="cat_Other"
    return cat
    

In [28]:
uniq_lab["group"]=uniq_lab.category.apply(to_Category)
uniq_lab.to_csv("files/unique_categories_grouped.csv",index=False)

In [29]:
# prioritise categorisation
apps['general_groups'] = apps['category']

apps.general_groups = apps["general_groups"].apply(to_Category)

In [30]:
apps.general_groups.value_counts()

cat_Property         57896
cat_Industry tag     56902
cat_Custom label     53936
cat_Game_Other       53015
cat_Tencent          49320
cat_Finance          25135
cat_Free             21827
cat_Game_RPG         18416
cat_Entertainment    17501
cat_Game_Skill       16684
cat_Service          13511
cat_Productivity     12765
cat_Family           11289
cat_Other             7513
cat_Communication     6540
cat_Health            5453
cat_Esoteric          5339
cat_Game_Action       4780
cat_Travel            4574
cat_Game_Sports       4041
cat_House             3269
cat_Shopping          3234
cat_unknown           2754
cat_Education         2360
cat_Game_Gambling      739
cat_Culture            666
cat_Information        337
cat_Beauty             147
Name: general_groups, dtype: int64

We probably have to merge some categories, but first let's test them. These are setup to have some separation power. This is not a true statement. More important, as same `app_id`s can have multiple labels, is the number of occurances in the data.

In [31]:
apps.head(2)

,app_id,label_id,category,general_groups
0,7324884708820027918,251,Finance,cat_Finance
1,-4494216993218550286,251,Finance,cat_Finance


In [32]:
apps = apps.drop(["label_id","category"], axis=1)
print(apps.head(2))

                app_id general_groups
0  7324884708820027918    cat_Finance
1 -4494216993218550286    cat_Finance


In [34]:
apps.to_csv("files/finalSets/app_categories_grouped.csv", index=False)